In [17]:
import numpy as np
import scipy as sc
import open3d as o3d
from math import sqrt
import matplotlib.pyplot as plt
from scipy.spatial import ConvexHull
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

In [18]:
result = []


for i in range(1, 11):
    name = str(i)
    PLY = f"./{name}.ply"
    mesh = o3d.io.read_triangle_mesh(PLY)
    mesh.compute_vertex_normals()

    pcd = o3d.geometry.PointCloud()
    pcd.points = mesh.vertices
    pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.01, max_nn=30))
    pcd.orient_normals_consistent_tangent_plane(100)
    p, a, v = perimeter(pcd)/DEPTH_UNIT, area(mesh)/DEPTH_UNIT**2, volume(pcd, name)/DEPTH_UNIT**3
    result.append((p, a, v))

In [22]:
result = np.array(result)
m = np.nanmean(result, axis=0)
for t in np.argwhere(np.isnan(result)):
    result[t[0], t[1]] = m[t[1]]
value = np.array([[np.pi * 40, np.pi * 840, np.pi * 4400] * result.shape[0]]).reshape(result.shape)
print(result)

[[  115.36159632  2040.85331465 13508.28552197]
 [  134.63810081  2135.7494002  14395.79846288]
 [  137.24029191  2254.34844507 15744.85747332]
 [  138.20170719  2489.68073282 16405.70121997]
 [  135.85536198  2226.6689118  15595.6775057 ]
 [  118.91547872  2090.25855754 13446.89242905]
 [  138.68947952  2477.73640707 16584.61082972]
 [  136.06529079  2193.86508415 15824.3503675 ]
 [  136.44271432  2380.11083176 16301.3703937 ]
 [  138.68947952  2477.73640707 16584.61082972]]


In [23]:
mean_absolute_error(value, result, multioutput='raw_values')

array([  10.75631141,  362.2370198 , 1754.37530767])

In [24]:
mean_absolute_percentage_error(value, result, multioutput='raw_values')

array([0.08559601, 0.13726622, 0.12691705])

In [8]:
DEPTH_UNIT = 0.001

# Perímetro

Se calcula el perimetro como la suma de las distancia Euclideana de los puntos de la frontera de la ulcera. Para ello se calculan los puntos que pertenecen a la envoltura convexa de la nube de puntos y se calcula la distancia entre ellos. 

- Fuente: Wound 3D Geometrical Feature Estimation Using Poisson Reconstruction

In [3]:
def perimeter(ulcer_pts):
    ulcer2d = np.asarray(ulcer_pts.points)[:,:2]
    ch = ConvexHull(ulcer2d)
    p = 0
    for edge in ch.simplices:
        p += euclidean_distances([ulcer2d[edge[0]]], [ulcer2d[edge[1]]])[0][0]
    return p

In [2]:
print("Perímetro:", perimeter(pcd) / DEPTH_UNIT, "mm")

NameError: name 'pcd' is not defined

# Área

El área se calcula como la suma de todos los triángulos de la malla utilizando la fórmula de Herón.

$A_t = \sqrt{s(s-a)(s-b)(s-c)}$

donde $s = \frac{P_t}{2}$ y $P_t$ es el perímetro del triángulo.

Entonces $A_M = \sum_{i = 0}^n A_{t_i}$ donde $M$ es la malla con $n$ triángulos.

In [4]:
def heron(p):
    S = euclidean_distances(p, p)
    SP = (S[0][1] + S[0][2] + S[1][2])/2
    return sqrt(SP * (SP - S[0][1]) * (SP - S[0][2]) * (SP - S[1][2]))

def area(ulcer_mesh):
    triangles = np.asarray(ulcer_mesh.triangles)
    points = np.asarray(ulcer_mesh.vertices)
    area = 0
    for t in triangles:
        pts = points[t]
        area += heron(pts)
    return area

def area_o3d(ulcer_mesh):
    return ulcer_mesh.get_surface_area()

In [ ]:
print("Area:",area(mesh) / DEPTH_UNIT**2, "mm2")

# Volumen

Para calcular el volumen se procede a generar la tapa de la ulcera utilizando spline cubico, luego se genera una triangulacion de Delaunay entre la tapa y la ulcera para luego calcular el volumen de cada piramide resultante de la triangulacion utilizando el area segun Heron y la distancia de punto a plano.

In [11]:
def get_plane(x: np.array, y: np.array, z: np.array):
    xy = y - x
    xz = z - x
    n = np.cross(xy, xz)
    d = np.dot(x, n)
    return np.concatenate((n, [-d]))

def pP_distance(p0: np.array, x: np.array, y: np.array, z: np.array):
    plane = get_plane(x, y, z)
    num = abs(np.dot(np.concatenate((p0, [1])), plane))
    den = sqrt(np.sum(plane[:3] ** 2))
    return num / den

def get_top(ulcer_pts, save_top = False, name=""):
    # obtengo los puntos del borde de la ulcera
    #points_boundary = np.asarray(pcd.select_by_index(ulcer_pts.compute_convex_hull()[1]).points)
    ulcer2d = np.asarray(ulcer_pts.points)[:,:2]
    ch = ConvexHull(ulcer2d)
    points_boundary = np.asarray(pcd.points)[ch.vertices]
    # inicializo un interpolador
    interpolate = sc.interpolate.CloughTocher2DInterpolator(points_boundary[:,:2], points_boundary[:,-1])
    #interpolate = sc.interpolate.LinearNDInterpolator(points_boundary[:,:2], points_boundary[:,-1])
    # calculo los maximos y los minimos de los puntos del borde para generar la tapa
    mins = np.min(points_boundary[:,:2], axis=0)
    maxs = np.max(points_boundary[:,:2], axis=0)
    x = np.linspace(mins[0], maxs[0], 100)
    y = np.linspace(mins[1], maxs[1], 100)
    x, y = np.meshgrid(x, y)
    points = np.dstack((x, y))
    # genero los puntos de la tapa usando la interpolacion
    points_3d = []
    for i in range(points.shape[0]):
        for j in range(points.shape[1]):
            points_3d.append([points[i][j][0], points[i][j][1], interpolate(points[i][j])[0]])
    points_3d = np.array(points_3d)
    points_3d = points_3d[~np.isnan(points_3d[:,-1])]
    top = o3d.geometry.PointCloud()
    top.points = o3d.utility.Vector3dVector(points_3d)

    if save_top:
        o3d.io.write_point_cloud(f"./{name}.ply", top)
    return top

def volume(ulcer_pts, name):
    points_3d = get_top(ulcer_pts, False, name)
    # luego hago la triangulacion de Delaunay en 3D
    delaunay = sc.spatial.Delaunay(np.concatenate((points_3d.points, np.asarray(ulcer_pts.points))))
    # se calcula el volumen de cada piramide
    volume = 0
    for pyramid in delaunay.simplices:
        pts = delaunay.points[pyramid]
        AB = heron(pts[1:])
        h = pP_distance(pts[0], pts[1], pts[2], pts[3])
        volume += (AB * h) / 3
    return volume

In [83]:
print("Volumen",volume(pcd, name) / DEPTH_UNIT**3, "mm3")

[230 200 170 106  22   6   5   1   0   7  23  48 107 171 325 388 477 604
 629 696 698 701 709 714 695 673 603 580 531]
Volumen 16310.635234263693 mm3
